In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt


#np.set_printoptions(precision=4)
import math
from utils import *

nS = 250
nA = 2
seed = 2184109
gamma = .9
mu = np.ones(nS) * 1/nS
river = River(nS, mu, gamma=gamma, small=5, large=1000, seed=seed)
tau = 1.
xi = np.ones(nS) * 1/nS
tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)



In [2]:
episodes = 10000
status_step = 10000
alpha = 0.25
alpha_tau = 0.003
results = {"Qs": [], "taus": [], "steps": 0}

Q = np.zeros((nS, nA))
while tau < 1.00001 and tau > 0:
    res = Q_learning(tmdp, Q, episodes=episodes, alpha=alpha, status_step=status_step)
    Q = res["Qs"][-1]
    visit_dist = res["visit_distributions"][-1]
    results["steps"] += episodes
    
    pi = get_policy(Q)

    r_s_a_xi = compute_r_s_a(tmdp.xi, tmdp.reward)
    r_s_a_p = compute_r_s_a(tmdp.P_mat, tmdp.reward)
    r_s_a = compute_r_s_a(tmdp.P_mat_tau, tmdp.reward)

    #d = compute_d(mu, tmdp.P_mat_tau, pi, tmdp.gamma)

    q_p = get_q_hat( tmdp.P_mat, r_s_a_p, tmdp.gamma, Q)
    q_xi = get_q_hat(tmdp.xi, r_s_a_xi, tmdp.gamma, Q)
    grad = compute_grad_j(pi, q_p, q_xi, visit_dist, tmdp.gamma)

    print("Gradient: ", grad)
    if grad < 0:
        tau = max(0, tau + alpha_tau * grad)
        results["Qs"].append(Q)
        results["taus"].append(tmdp.tau)
        print("Updating tau from {} to {}".format(tmdp.tau, tau))
        tmdp.update_tau(tau)
    

Gradient:  -0.20891707317073177
Updating tau from 1.0 to 0.9993732487804878
Gradient:  -0.20405853658536593
Updating tau from 0.9993732487804878 to 0.9987610731707317
Gradient:  -0.16033170731707322
Updating tau from 0.9987610731707317 to 0.9982800780487805
Gradient:  -0.1943414634146342
Updating tau from 0.9982800780487805 to 0.9976970536585366
Gradient:  -0.18948292682926837
Updating tau from 0.9976970536585366 to 0.9971286048780488
Gradient:  -0.18948292682926837
Updating tau from 0.9971286048780488 to 0.996560156097561
Gradient:  -0.21377560975609763
Updating tau from 0.996560156097561 to 0.9959188292682927
Gradient:  -0.20891707317073177
Updating tau from 0.9959188292682927 to 0.9952920780487805
Gradient:  -0.1408975609756098
Updating tau from 0.9952920780487805 to 0.9948693853658537
Gradient:  -0.11660487804878052
Updating tau from 0.9948693853658537 to 0.9945195707317074
Gradient:  -0.18948292682926837
Updating tau from 0.9945195707317074 to 0.9939511219512196
Gradient:  -0.1894

In [3]:
print(results["steps"])
Q_0 = np.zeros((nS, nA))
Qs_0 = Q_learning(tmdp, Q_0, episodes=results["steps"], alpha=alpha, status_step=results["steps"]/20)["Qs"]
Q_0 = Qs_0[-1]
print("Q0: ", Q_0)



690000
Q0:  [[5.00000000e+01 4.34710715e+01]
 [4.50000000e+01 3.97903949e+01]
 [4.05000000e+01 3.54230703e+01]
 [3.64500000e+01 3.18543044e+01]
 [3.28050000e+01 2.83884808e+01]
 [2.95245000e+01 2.62039825e+01]
 [2.65720500e+01 2.32730956e+01]
 [2.39148450e+01 2.11065026e+01]
 [2.15233605e+01 1.92735898e+01]
 [1.93710244e+01 1.73280854e+01]
 [1.74339220e+01 1.54630012e+01]
 [1.56905298e+01 1.38151934e+01]
 [1.41214768e+01 1.26954099e+01]
 [1.27093291e+01 1.10539843e+01]
 [1.14383962e+01 1.02609123e+01]
 [1.02945566e+01 9.06261561e+00]
 [9.26510094e+00 8.30007832e+00]
 [8.33859085e+00 7.34319107e+00]
 [7.50473176e+00 6.62825610e+00]
 [6.75425859e+00 6.03002836e+00]
 [6.07883273e+00 5.24291424e+00]
 [5.47094946e+00 4.76821281e+00]
 [4.92385451e+00 4.41375224e+00]
 [4.43146906e+00 3.91378722e+00]
 [3.98832215e+00 3.52788055e+00]
 [3.58948994e+00 3.14479463e+00]
 [3.23054094e+00 2.83762668e+00]
 [2.90748685e+00 2.59047287e+00]
 [2.61673817e+00 2.35706954e+00]
 [2.35506435e+00 2.07228553e+00

In [4]:
Q_star = bellman_optimal_q(tmdp.P_mat, tmdp.reward, tmdp.gamma)["Q"]

In [5]:
print(results["steps"])
finalQ = results["Qs"][-1]
print(finalQ)
print(get_policy(finalQ))

690000
[[  37.0203555    30.20127698]
 [  30.98790736   21.24109371]
 [  26.17198644   21.86135414]
 [  22.94610772   18.55108569]
 [  19.55720619   16.34362331]
 [  16.58249655   16.36349049]
 [  15.81545809   12.89642478]
 [  13.52239209   12.25560171]
 [  11.7949077    11.03280998]
 [  10.74900587   10.64497872]
 [   9.66787851    9.03731234]
 [   8.80808566    7.40131212]
 [   8.32510849    5.300793  ]
 [   6.95567173    5.90944392]
 [   6.18233373    5.01937957]
 [   5.5811988     4.52681249]
 [   4.79428407    4.26249895]
 [   4.17512132    3.41767748]
 [   3.78248694    2.73077178]
 [   3.6721349     2.56390872]
 [   5.16369739    2.4470047 ]
 [   5.74265518    2.23599954]
 [   3.39074563    2.35655559]
 [   2.50207467    2.31055258]
 [   2.22566914    3.18428508]
 [   3.25164497    2.377534  ]
 [   2.6078802     2.402122  ]
 [   2.36179647    2.36040124]
 [   2.37834173    2.38240092]
 [   2.27241903    2.25414072]
 [   2.19234936    2.19301977]
 [   2.23862906    2.23951892]
 

In [6]:
# Evaluating performance of the final policy
J_final = get_expected_avg_reward(tmdp.P_mat, get_policy(finalQ), tmdp.reward, tmdp.gamma, tmdp.mu)
J_star = get_expected_avg_reward(tmdp.P_mat, get_policy(Q_star), tmdp.reward, tmdp.gamma, tmdp.mu)
J_0 = get_expected_avg_reward(tmdp.P_mat, get_policy(Q_0), tmdp.reward, tmdp.gamma, tmdp.mu)

print("J_final: ", J_final)
print("J_star: ", J_star)
print("J_0: ", J_0)

J_final:  13.81279926460114
J_star:  14.209656304397933
J_0:  3.6438356164310512
